# OSM Highway Extraction Example

This notebook demonstrates how to extract highway networks from OpenStreetMap
using the OSM Highways processor.

## Features
- Download regional OSM data from Geofabrik
- Extract highways for any AOI
- Filter by highway type
- Export to multiple formats
- Track data freshness

In [1]:
from pathlib import Path
import geopandas as gpd
import matplotlib.pyplot as plt

from geoworkflow.processors.extraction.osm_highways import OSMHighwaysProcessor
from geoworkflow.schemas.osm_highways_config import OSMHighwaysConfig
from geoworkflow.utils.geofabrik_utils import list_cached_pbfs

import logging
logging.getLogger('geoworkflow').setLevel(logging.DEBUG)

In [ ]:
import geopandas as gpd 
abf = gpd.read_file("../data/00_source/boundaries/agglomerations.gpkg")
abf.head(4).drop("geometry",axis=1).to_csv("agglomerations_preview.txt")

# Define AOI

In [ ]:
from geoworkflow.schemas.config_models import AOIConfig
from geoworkflow.processors.aoi.processor import AOIProcessor

# Define AOI output path
aoi_file = Path("../data/aoi/accra_aoi.geojson")
#aoi_file = Path("../data/aoi/accra_sample_aoi.geojson")

# Create AOI configuration for Accra
aoi_config = AOIConfig(
    input_file=Path("../data/00_source/boundaries/agglomerations.gpkg"),
    country_name_column="Agglomeration_Name",
    countries=["Accra"],
    buffer_km=0,
    dissolve_boundaries=False,
    output_file=aoi_file
)

# Create and run the processor
aoi_processor = AOIProcessor(aoi_config)
aoi_result = aoi_processor.process()

# Check results
if aoi_result.success:
    print(f"✅ {aoi_result.message}")
    print(f"Processing time: {aoi_result.elapsed_time:.2f}s")
    print(f"Output: {aoi_file}")
else:
    print(f"❌ Failed: {aoi_result.message}")

## Example 1: Basic Extraction

Extract all highways for a city AOI.

In [ ]:
# Setup paths
aoi_file = Path("../data/aoi/accra_aoi.geojson")

output_dir = Path("../data/01_extracted/accra_highways")
output_dir.mkdir(parents=True, exist_ok=True)

# Create config
config = OSMHighwaysConfig(
    aoi_file=aoi_file,
    output_dir=output_dir,
    highway_types="all",  # Extract all highway types
    include_attributes=["highway", "name", "surface", "lanes"],
    export_format="geojson",
    force_redownload= True,
    max_cache_age_days=30  # Warn if data >30 days old
)

# Run extraction
processor = OSMHighwaysProcessor(config)
result = processor.process()

print(f"Success: {result.success}")
print(f"Extracted: {result.processed_count} highway segments")
print(f"Output: {processor.output_file}")

In [ ]:
# Process all agglomerations in Ghana and Togo
config = OSMHighwaysConfig(
    aoi_file="africapolis",
    country=["GHA","TGO"],
    output_dir=Path("../data/extracted/ghanatogo_batch")
)

processor = OSMHighwaysProcessor(config)
result = processor.process()
print(result)

/Users/juancheeto/miniconda3/envs/geoworkflow/lib/python3.10/site-packages/pyrosm/networks.py:37: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  edges, nodes = prepare_geodataframe(
✗ Takoradi: No highways found
✗ Cape Coast: No highways foun

## Example 3: Visualize Results

In [ ]:
# Load results
highways = gpd.read_file(processor.output_file)
aoi = gpd.read_file(aoi_file)

# Plot
fig, ax = plt.subplots(figsize=(12, 12))
aoi.boundary.plot(ax=ax, color='red', linewidth=2, label='AOI')

# Color by highway type
highway_colors = {
    'motorway': 'darkblue',
    'trunk': 'blue',
    'primary': 'green',
    'secondary': 'orange',
    'tertiary': 'yellow',
    'residential': 'lightgray'
}

for htype, color in highway_colors.items():
    subset = highways[highways['highway'] == htype]
    if len(subset) > 0:
        subset.plot(ax=ax, color=color, linewidth=1, label=htype)

plt.legend()
plt.title("Extracted Highway Network")
plt.show()

## Example 4: Check Cache Status

In [ ]:
cache_dir = Path.home() / ".geoworkflow" / "osm_cache"

print("Cached PBF files:")
for region, meta in list_cached_pbfs(cache_dir):
    print(f"  {region}: {meta.file_size_mb:.1f} MB, {meta.age_days()} days old")
    print(f"    Downloaded: {meta.download_date}")
    print(f"    Source: {meta.geofabrik_url}")